SpaceX launch anlysis
Analyzing SpaceX launch data from the public API to understand launch success rates by rocket, pad and year

#1 import libraries and fetch baseline launch data from API and store in pandas dataframe

In [ ]:
import requests
import pandas as pd
import json
# API endpoint for all launches
url = "https://api.spacexdata.com/v4/launches"

# Fetch data
response = requests.get(url)
if response.status_code == 200:
    launches = response.json()
   
else:
    print("Error fetching data:", response.status_code)
    launches = []

# Extract relevant fields
launch_data = []
for launch in launches:
    launch_data.append({
        "launch_name": launch["name"],
        "date": launch["date_utc"],
        "success": launch["success"],
        "rocket_id": launch["rocket"],
        "payloads": len(launch["payloads"]),
        "launchpad_id": str(launch["launchpad"])
    })

# Convert to DataFrame
df = pd.DataFrame(launch_data)

df.to_csv('data/spacex_launches.csv', index=False)

#2 Fetch rocket data from rocket API and left join on rocket_id. adding actual rocket name to dataframe

In [ ]:
all_rockets_url = f"https://api.spacexdata.com/v4/rockets"

all_rocket_res = requests.get(all_rockets_url)

all_rockets = all_rocket_res.json()

rocket_df = pd.DataFrame([{
    "rocket_id": rocket["id"]
    ,"rocket_name": rocket["name"]
} for rocket in all_rockets])

df = df.merge(rocket_df, left_on="rocket_id", right_on="rocket_id", how="left")

#3 Fetch launchpad data and left join on launchpad id, adding a launchpad name and location to the data.

In [ ]:
launchpads_url = "https://api.spacexdata.com/v4/launchpads"

launchpads_res = requests.get(launchpads_url)

launchpads = launchpads_res.json()

launchpad_df = pd.DataFrame([{
    "launchpad_id": pad["id"]
    ,"launchpad_name": pad["name"]
    ,"launchpad_location": pad["locality"]
} for pad in launchpads ])

df = df.merge(launchpad_df, left_on="launchpad_id", right_on="launchpad_id", how="left")

analysis: success rate by rocket, launchpad and year

In [ ]:
df["date"] = pd.to_datetime(df["date"])

df["year"] = df["date"].dt.year

grouped_df = df.groupby(["rocket_name","launchpad_name","year"])

summary = grouped_df["success"].agg(
    total_launches="count"
    ,successful_launches = lambda x: x.sum()
    ,success_rate=lambda x:(x.sum() / x.count()) * 100
).reset_index()

summary = summary.sort_values(by="success_rate", ascending=False)
print(summary)

In [ ]:
summary.to_csv('data/spacex_launches.csv', index=False)

conclusion
-falcon 9 
-most launches occur at CCSFS SLC 40

next steps
- payload data could be added
- csv could be visulized and presented over years